In [1]:
import csv
import json
import os

import pandas as pd

In [2]:
KEY1 = 'correct'
KEY2 = 'horse'

SERVER = os.getenv("SERVER", "https://testing.es.data61.xyz")

# Scenario

There are three parties named Alice, Bob, and Charlie, each holding a dataset of about 3200 records. They know that they have some entities in common, but with incomplete overlap. The common features describing those entities are given name, surname, date of birth, and phone number.

They all have some additional information about those entities in their respective datasets, Alice has a person's gender, Bob has their city, and Charlie has their income. They wish to create a table for analysis: each row has a gender, city, and income, but they don't want to share any additional information. They can use Anonlink to do this in a privacy-preserving way (without revealing given names, surnames, dates of birth, and phone numbers).

## Alice, Bob, and Charlie: agree on secret keys and a linkage schema

They keep the keys to themselves, but the schema may be revealed to the analyst.

In [3]:
print(f'keys: {KEY1}, {KEY2}')

keys: correct, horse


In [4]:
with open('data/schema_ABC.json') as f:
    print(f.read())


{
  "version": 2,
  "clkConfig": {
    "l": 1024,
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
      "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
      "info": "c2NoZW1hX2V4YW1wbGU=",
      "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "id",
      "ignored": true
    },
    {
      "identifier": "givenname",
      "format": {
        "type": "string",
        "encoding": "utf-8"
      },
      "hashing": {
        "ngram": 2,
        "positional": false,
        "strategy": {"k": 15}
      }
    },
    {
      "identifier": "surname",
      "format": {
        "type": "string",
        "encoding": "utf-8"
      },
      "hashing": {
        "ngram": 2,
        "positional": false,
        "strategy": {"k": 15}
      }
    },
    {
      "identifier": "dob",
      "format": {
        "type": "string",
        "encoding": "utf-8"
      },
      "hashing": {
        "ngram": 2,
        "positional":

# Sneak peek at input data
### Alice

In [5]:
pd.read_csv('data/dataset-alice.csv').head()

,id,givenname,surname,dob,phone number,gender
0,0,tara,hilton,27-08-1941,08 2210 0298,male
1,3,saJi,vernre,22-12-2972,02 1090 1906,mals
2,7,sliver,paciorek,NaN,NaN,mals
3,9,ruby,george,09-05-1939,07 4698 6255,male
4,10,eyrinm,campbell,29-1q-1983,08 299y 1535,male


### Bob

In [6]:
pd.read_csv('data/dataset-bob.csv').head()

,id,givenname,surname,dob,phone number,city
0,3,zali,verner,22-12-1972,02 1090 1906,perth
1,4,samuel,tremellen,21-12-1923,03 3605 9336,melbourne
2,5,amy,lodge,16-01-1958,07 8286 9372,canberra
3,7,oIji,pacioerk,10-02-1959,04 4220 5949,sydney
4,10,erin,kampgell,29-12-1983,08 2996 1445,perth


## Charlie

In [7]:
pd.read_csv('data/dataset-charlie.csv').head()

,id,givenname,surname,dob,phone number,income
0,1,joshua,arkwright,16-02-1903,04 8511 9580,70189.446
1,3,zal:,verner,22-12-1972,02 1090 1906,50194.118
2,7,oliyer,paciorwk,10-02-1959,04 4210 5949,31750.993
3,8,nacoya,ranson,17-08-1925,07 6033 4580,102446.131
4,10,erih,campbell,29-12-1i83,08 299t 1435,331476.599


## Analyst: create the project

The analyst keeps the result token to themselves. The three update tokens go to Alice, Bob and Charlie. The project ID is known by everyone.

In [8]:
!clkutil create-project --server $SERVER --type groups --schema data/schema_ABC.json --parties 3 --output credentials.json

with open('credentials.json') as f:
    credentials = json.load(f)
    project_id = credentials['project_id']
    result_token = credentials['result_token']
    update_token_alice = credentials['update_tokens'][0]
    update_token_bob = credentials['update_tokens'][1]
    update_token_charlie = credentials['update_tokens'][2]

Project created


## Alice: hash the data and upload it to the server
The data is hashed according to the schema and the keys. Alice's update token is needed to upload the hashed data. No PII is uploaded to the service—only the hashes.

In [9]:
!clkutil hash data/dataset-alice.csv $KEY1 $KEY2 data/schema_ABC.json dataset-alice-hashed.json --check-header false

generating CLKs: 100%|█| 3.23k/3.23k [00:00<00:00, 16.2kclk/s, mean=372, std=34.9]
CLK data written to dataset-alice-hashed.json


In [10]:
!clkutil upload --server $SERVER --apikey $update_token_alice --project $project_id dataset-alice-hashed.json

{"message": "Updated", "receipt_token": "4bf04c3205064a79a38e53f692d20fbdd37de027c15509fe"}

## Bob: hash the data and upload it to the server

In [11]:
!clkutil hash data/dataset-bob.csv $KEY1 $KEY2 data/schema_ABC.json dataset-bob-hashed.json --check-header false

generating CLKs: 100%|█| 3.24k/3.24k [00:00<00:00, 16.1kclk/s, mean=372, std=35.3]
CLK data written to dataset-bob-hashed.json


In [12]:
!clkutil upload --server $SERVER --apikey $update_token_bob --project $project_id dataset-bob-hashed.json

{"message": "Updated", "receipt_token": "3e7d3aa6f6bf8864c6c15a581bc8dd1b98da64441b2ab696"}

## Charlie: hash the data and upload it to the server

In [13]:
!clkutil hash data/dataset-charlie.csv $KEY1 $KEY2 data/schema_ABC.json dataset-charlie-hashed.json --check-header false

generating CLKs: 100%|█| 3.26k/3.26k [00:00<00:00, 15.8kclk/s, mean=372, std=34.8]
CLK data written to dataset-charlie-hashed.json


In [14]:
!clkutil upload --server $SERVER --apikey $update_token_charlie --project $project_id dataset-charlie-hashed.json

{"message": "Updated", "receipt_token": "00f8ffb0965e93a5b8e7a479ec71387d08d430b05f4c75c9"}

## Analyst: start the linkage run

This will start the linkage computation. We will wait a little bit and then retrieve the results.

In [15]:
!clkutil create --server $SERVER --project $project_id --apikey $result_token --threshold 0.7 --output=run-credentials.json

with open('run-credentials.json') as f:
    run_credentials = json.load(f)
    run_id = run_credentials['run_id']

## Analyst: retreve the results

In [16]:
!clkutil results --server $SERVER --project $project_id --apikey $result_token --run $run_id --watch --output linkage-output.json

State: running
Stage (2/3): compute similarity scores
State: running
Stage (3/3): compute output
State: completed
Stage (3/3): compute output
Received result


In [17]:
with open('linkage-output.json') as f:
    linkage_output = json.load(f)
    linkage_groups = linkage_output['groups']

## Everyone: make table of interesting information

We use the linkage result to make a table of genders, cities, and incomes without revealing any other PII.

In [18]:
with open('data/dataset-alice.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    genders = tuple(row[-1] for row in r)
    
with open('data/dataset-bob.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    cities = tuple(row[-1] for row in r)
    
with open('data/dataset-charlie.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    incomes = tuple(row[-1] for row in r)

In [19]:
table = []
for group in linkage_groups:
    row = [''] * 3
    for i, j in group:
        row[i] = [genders, cities, incomes][i][j]
    if sum(map(bool, row)) > 1:
        table.append(row)
pd.DataFrame(table, columns=['gender', 'city', 'income']).head(10)

,gender,city,income
0,,peGh,395273.665
1,,melbourne,157033.920
2,,pertb,323383.650
3,,syd1e7y,79745.538
4,,perth,28019.494
5,,canberra,78961.675
6,female,brisnane,
7,female,melbourne,
8,male,melbourne,
9,,melbourne,68548.966


In [20]:
linkage_groups[:15]

[[[1, 687], [2, 711]],
 [[1, 2298], [2, 2313]],
 [[0, 1040], [2, 1057]],
 [[1, 2101], [2, 2120]],
 [[1, 281], [2, 280]],
 [[1, 798], [2, 834]],
 [[1, 123], [2, 129]],
 [[1, 2923], [2, 2815]],
 [[0, 2938], [1, 2960]],
 [[0, 1376], [1, 1305]],
 [[0, 804], [1, 946]],
 [[1, 981], [2, 1027]],
 [[0, 2413], [2, 2437]],
 [[0, 856], [2, 879], [1, 834]],
 [[0, 2167], [1, 2152]]]

# Sneak peek at the result

We obviously can't do this in a real-world setting, but let's view the linkage using the PII. If the IDs match, then we are correct.

In [21]:
with open('data/dataset-alice.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    dataset_alice = tuple(r)
    
with open('data/dataset-bob.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    dataset_bob = tuple(r)
    
with open('data/dataset-charlie.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    dataset_charlie = tuple(r)

In [22]:
table = []
for group in linkage_groups:
    for i, j in sorted(group):
        table.append([dataset_alice, dataset_bob, dataset_charlie][i][j])
    table.append([''] * 6)
    
pd.DataFrame(table, columns=['id', 'given name', 'surname', 'dob', 'phone number', 'non-linking']).head(30)

,id,given name,surname,dob,phone number,non-linking
0,2916,zes5ida,passmore,15-93-1938,02 7596 9493,peGh
1,2916,jessjca,passmore,15-03-1937,02 7586 9393,395273.665
2,,,,,,
3,7344,tara,mazurek,04-10-1955,04 2775 5157,melbourne
4,7344,etq,masurem,04-10-1955,04 2775 4157,157033.920
5,,,,,,
6,3861,tamiem,esdunary,31-11-1920,08 9101 8215,male
7,3861,damien,stedunary,31-11-1910,08 9101 8215,
8,,,,,,
9,6816,danielle,grin,01-01-1008,04 7622 3931,pertb
